# Using Reddit's API for Predicting Comments

In this project, we will practice two major skills. Collecting data via an API request and then building a binary predictor.

As we discussed in week 2, and earlier today, there are two components to starting a data science problem: the problem statement, and acquiring the data.

For this article, your problem statement will be: _What characteristics of a post on Reddit contribute most to what subreddit it belongs to?_

Your method for acquiring the data will be scraping threads from at least two subreddits. 

Once you've got the data, you will build a classification model that, using Natural Language Processing and any other relevant features, predicts which subreddit a given post belongs to.

### Scraping Thread Info from Reddit.com

#### Set up a request (using requests) to the URL below. 

*NOTE*: Reddit will throw a [429 error](https://httpstatuses.com/429) when using the following code:
```python
res = requests.get(URL)
```

This is because Reddit has throttled python's default user agent. You'll need to set a custom `User-agent` to get your request to work.
```python
res = requests.get(URL, headers={'User-agent': 'YOUR NAME Bot 0.1'})
```

#### Use `res.json()` to convert the response into a dictionary format and set this to a variable. 

```python
data = res.json()
```

In [1]:
import pandas as pd
import numpy as np

import datetime as dt
import time
import requests
import json

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
np.random.seed(42)

import string
import re 
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

/Users/dr.holtphd/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### Getting more results

By default, Reddit will give you the top 25 posts:

```python
print(len(data['data']['children']))
```

If you want more, you'll need to do two things:
1. Get the name of the last post: `data['data']['after']`
2. Use that name to hit the following url: `http://www.reddit.com/r/boardgames.json?after=THE_AFTER_FROM_STEP_1`
3. Create a loop to repeat steps 1 and 2 until you have a sufficient number of posts. 

*NOTE*: Reddit will limit the number of requests per second you're allowed to make. When you create your loop, be sure to add the following after each iteration.

```python
time.sleep(3) # sleeps 3 seconds before continuing```

This will throttle your loop and keep you within Reddit's guidelines. You'll need to import the `time` library for this to work!

### Save your results as a CSV
You may do this regularly while scraping data as well, so that if your scraper stops or your computer crashes, you don't lose all your data.

In [0]:
def query_pushshift(subreddit, kind='submission', skip=2, times=150, 
                    subfield = ['title', 'selftext', 'subreddit', 'created_utc', 'author', 'num_comments', 'score', 'is_self']):

    stem = "https://api.pushshift.io/reddit/search/{}/?subreddit={}&size=500".format(kind, subreddit)
    mylist = []
    
    for x in range(1, times):
        
        URL = "{}&after={}d".format(stem, skip * x)
        print(URL)
        response = requests.get(URL)
        assert response.status_code == 200
        mine = response.json()['data']
        df = pd.DataFrame.from_dict(mine)
        mylist.append(df)
        time.sleep(0.25)
        
    full = pd.concat(mylist)
    
    if kind == "submission":
        
        full = full[subfield]
        
        full = full.drop_duplicates()
        
        full = full.loc[full['is_self'] == True]
        
    def get_date(created):
        return dt.date.fromtimestamp(created)
    
    _timestamp = full["created_utc"].apply(get_date)
    
    full['timestamp'] = _timestamp

    print(full.shape)
    
    return full 

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

!ls

In [6]:
lpt_df = pd.read_csv('/Users/dr.holtphd/Documents/ga_portfolio/project-3/LifeProTips')
ulpt_df = pd.read_csv('/Users/dr.holtphd/Documents/ga_portfolio/project-3/UnethicalLifeProTips')

In [7]:
lpt_df.head()

Unnamed: 0                                              title                                         selftext    subreddit  created_utc               author  num_comments  score  is_self   timestamp
0           0  If you are going out to drink and have work th...                                        [removed]  LifeProTips   1536257587             ph120299             2      1     True  2018-09-06
1           1  LPT: For a cleaner wipe when pooping, firmly p...                                        [removed]  LifeProTips   1536257946      AncientCodpiece             1      1     True  2018-09-06
2           2  LPT: Wedding Tip: Always check Craigslist/Face...                                        [removed]  LifeProTips   1536257961        60SecondBoost             1      1     True  2018-09-06
3           3  LPT: If you ever get a parking ticket, save th...  This way it looks like you already got ticketed  LifeProTips   1536258048           cigar_dude             9      1     True  2018-09-06
4           4  LPT: when stopping for gas on a long drive, go...                                        [removed]  LifeProTips   1536258230  Hannarks_the_Hunter             2      1     True  2018-09-06

In [8]:
ulpt_df.head()

Unnamed: 0                                              title   selftext             subreddit  created_utc        author  num_comments  score  is_self   timestamp
0           0  ULPT If the dumbass mods of a sub require subs...  [removed]  UnethicalLifeProTips   1536258969  WitnessMeIRL             1      1     True  2018-09-06
1           1  ULPT: Buy a bunch of cute lady socks. Offer th...        NaN  UnethicalLifeProTips   1536259519   SalcaPaneer             3      1     True  2018-09-06
2           2  ULPT: If you live in a WG and your roommates s...        NaN  UnethicalLifeProTips   1536260003         jakif             0      1     True  2018-09-06
3           3  ULPT: If you live in a shared flat and your ro...        NaN  UnethicalLifeProTips   1536261695         jakif             9      1     True  2018-09-06
4           4  Want free food, "accidentally" drop it and mos...  [removed]  UnethicalLifeProTips   1536261701    Orange5050             1      1     True  2018-09-06

In [9]:
ulpt_df.shape

(23975, 10)

In [10]:
# Use half the data from lpt - random sample from the data

lptdf = lpt_df.sample(n=23975)

In [11]:
lptdf.shape

(23975, 10)

In [12]:
lptdf.head()

Unnamed: 0                                              title                                           selftext    subreddit  created_utc       author  num_comments  score  is_self   timestamp
23443          15  LPT: Tuesday’s @ 3:00 pm, six weeks before you...                                          [removed]  LifeProTips   1526069464   mmmikaykay             0      1     True  2018-05-11
17631          38  LPT: Keep a piece of shrink tubing on electric...  Cords that get mistreated a lot, power tools e...  LifeProTips   1528670760    lhj734017             6      1     True  2018-06-10
39490          71  If dumping / rinsing a lot of grimy stuff in t...                                          [removed]  LifeProTips   1519147579  DemiDualism             1      1     True  2018-02-20
50089         225  Write the brand and color of the garment on th...                                                NaN  LifeProTips   1514407258    [deleted]             1      1     True  2017-12-27
11489          76  LPT: If you have balloons that need to be defl...                                                NaN  LifeProTips   1531442400    smitty1ky             4      1     True  2018-07-12

In [13]:
#append dataframes for ethical and life protips

reddit = lptdf.append(ulpt_df)

In [14]:
reddit.head(10)

Unnamed: 0                                              title                                           selftext    subreddit  created_utc        author  num_comments  score  is_self   timestamp
23443          15  LPT: Tuesday’s @ 3:00 pm, six weeks before you...                                          [removed]  LifeProTips   1526069464    mmmikaykay             0      1     True  2018-05-11
17631          38  LPT: Keep a piece of shrink tubing on electric...  Cords that get mistreated a lot, power tools e...  LifeProTips   1528670760     lhj734017             6      1     True  2018-06-10
39490          71  If dumping / rinsing a lot of grimy stuff in t...                                          [removed]  LifeProTips   1519147579   DemiDualism             1      1     True  2018-02-20
50089         225  Write the brand and color of the garment on th...                                                NaN  LifeProTips   1514407258     [deleted]             1      1     True  2017-12-27
11489          76  LPT: If you have balloons that need to be defl...                                                NaN  LifeProTips   1531442400     smitty1ky             4      1     True  2018-07-12
2949          204  LPT: When shopping at a mall, take a photo of ...                                       &amp;#x200B;  LifeProTips   1535132879  clariceannej             5      1     True  2018-08-24
49940          76  LPT: When cutting your finger/toe nails blink ...                                          [removed]  LifeProTips   1514332814      bdawg439             1      1     True  2017-12-26
48293          22  LPT If you aren't sure whether or not you want...                                                NaN  LifeProTips   1515009800     RyukAtari             3     11     True  2018-01-03
31088          68  LPT: the government might have some free money...                                          [removed]  LifeProTips   1522801965      funcomfy             0      1     True  2018-04-03
50025         161  LPT: When at a concert or loud event and need ...                                          [removed]  LifeProTips   1514388676       noel926             1      1     True  2017-12-27

In [15]:
reddit.shape

(47950, 10)

## Preprocessing Text for NLP

In [16]:
# code target variables as 1 or 0

#reddit['target'] = reddit['subreddit'].replace(['LifeProTips',"UnethicalLifeProTips"], 1, 0)

#reddit['target'] = [1 if reddit.loc[i,'subreddit'] == 'LifeProTips' else 0 for i in range(reddit.shape[0])]

def convert_target(x):
    if x == 'LifeProTips':
        return 1
    if x == 'UnethicalLifeProTips':
        return 0
    
    
reddit['target'] = reddit['subreddit'].apply(convert_target)

In [17]:
reddit.iloc[21790:, 10]

15283    1
51850    1
29169    1
54963    1
38776    1
9698     1
27613    1
51812    1
44759    1
56296    1
25137    1
36857    1
10091    1
1889     1
4041     1
39608    1
59561    1
7467     1
40548    1
5018     1
51594    1
8013     1
39251    1
16214    1
12929    1
5667     1
36416    1
27722    1
19358    1
44492    1
1337     1
36629    1
41255    1
58989    1
54513    1
55064    1
43996    1
53706    1
12250    1
58349    1
11942    1
52688    1
19600    1
25341    1
6594     1
40217    1
15477    1
21194    1
26387    1
45935    1
20189    1
20626    1
1717     1
34167    1
951      1
1446     1
27910    1
29010    1
26242    1
15312    1
18564    1
30605    1
36341    1
25719    1
7249     1
25694    1
8398     1
4955     1
22845    1
20073    1
43694    1
32377    1
43991    1
23231    1
779      1
22071    1
41365    1
46944    1
47973    1
20496    1
2398     1
8475     1
29522    1
49573    1
16966    1
38662    1
57602    1
10173    1
31640    1
49771    1
7793     1

In [18]:
exclude = set(string.punctuation)
def remove_punctuation(x):
    """
    Helper function to remove punctuation from a string
    x: any string
    """
    try:
        x = ''.join(ch for ch in x if ch not in exclude)
    except:
        pass
    return x
# Apply the function to the DataFrame
reddit['work'] = reddit['title'].apply(remove_punctuation)

In [19]:
reddit['work'].head()

23443    LPT Tuesday’s  300 pm six weeks before your fl...
17631    LPT Keep a piece of shrink tubing on electrica...
39490    If dumping  rinsing a lot of grimy stuff in th...
50089    Write the brand and color of the garment on th...
11489    LPT If you have balloons that need to be defla...
Name: work, dtype: object

In [20]:
reddit.head()

Unnamed: 0                                              title                                           selftext    subreddit  created_utc       author  num_comments  score  is_self   timestamp  target                                               work
23443          15  LPT: Tuesday’s @ 3:00 pm, six weeks before you...                                          [removed]  LifeProTips   1526069464   mmmikaykay             0      1     True  2018-05-11       1  LPT Tuesday’s  300 pm six weeks before your fl...
17631          38  LPT: Keep a piece of shrink tubing on electric...  Cords that get mistreated a lot, power tools e...  LifeProTips   1528670760    lhj734017             6      1     True  2018-06-10       1  LPT Keep a piece of shrink tubing on electrica...
39490          71  If dumping / rinsing a lot of grimy stuff in t...                                          [removed]  LifeProTips   1519147579  DemiDualism             1      1     True  2018-02-20       1  If dumping  rinsing a lot of grimy stuff in th...
50089         225  Write the brand and color of the garment on th...                                                NaN  LifeProTips   1514407258    [deleted]             1      1     True  2017-12-27       1  Write the brand and color of the garment on th...
11489          76  LPT: If you have balloons that need to be defl...                                                NaN  LifeProTips   1531442400    smitty1ky             4      1     True  2018-07-12       1  LPT If you have balloons that need to be defla...

In [0]:
# Remove ULPT: from title 
# remove LPT: from title

In [0]:
# Tokenize the titles

## NLP

#### Use `CountVectorizer` or `TfidfVectorizer` from scikit-learn to create features from the thread titles and descriptions (NOTE: Not all threads have a description)
- Examine using count or binary features in the model
- Re-evaluate your models using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [21]:
X = reddit['work']
y = reddit['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)

In [22]:
cvec1 = CountVectorizer(stop_words='english', ngram_range=(2,2))  
cvec2 = CountVectorizer(stop_words='english', ngram_range=(5,5))
cvec3 = CountVectorizer(stop_words='english', ngram_range=(10,10))

In [23]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape


((35962,), (35962,), (11988,), (11988,))

In [24]:
cvec1.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [25]:
cvec1.transform(X_train)
cvec1.transform(X_test)

<11988x262358 sparse matrix of type '<class 'numpy.int64'>'
	with 62469 stored elements in Compressed Sparse Row format>

In [26]:
cvec1_df = pd.DataFrame(cvec1.get_feature_names())

In [27]:
cvec1_df.columns = ['feature1'] 

In [28]:
type(cvec1.transform(X_train))

scipy.sparse.csr.csr_matrix

In [29]:
cvec1_df.head()

feature1
0       000 followed
1           000 time
2  0000 wwwredditcom
3           001 work
4              01 10

In [30]:
cvec1_df['feature1'].value_counts().tail()

infamous red          1
want protect          1
notify school         1
learn bake            1
dance specifically    1
Name: feature1, dtype: int64

In [32]:


cvec1.fit(X_train)
cvtrans1_text_train = cvec1.transform(X_train)
cvtrans1_text_test = cvec1.transform(X_test)

cv11_df = pd.DataFrame(cvtrans1_text_train.todense(), columns = cvec1.get_feature_names())

In [33]:
cvec2.fit(X_train)
cvtrans2_text_train = cvec2.transform(X_train)
cvtrans2_text_test = cvec2.transform(X_test)



In [34]:
cvec2_df = pd.DataFrame(cvec2.get_feature_names())
cvec2_df.columns = ['feature2']
cvec2_df.head()

feature2
0        000 followed digits postal code
1  0000 wwwredditcom hosts file computer
2          001 work slavery getting paid
3         01 cooked properly youve saved
4         01 taken account help scammers

In [35]:
cvec2_df['feature2'].value_counts().tail()
cv22_df = pd.DataFrame(cvtrans2_text_train.todense(), columns = cvec2_df)

In [38]:
cvec3.fit(X_train)
cvtrans3_text_train = cvec3.transform(X_train)
cvtrans3_text_test = cvec3.transform(X_test)

In [39]:
cvec3_df = pd.DataFrame(cvec3.get_feature_names())
cvec3_df.columns = ['feature3']
cvec3_df.head()

feature3
0  0000 wwwredditcom hosts file computer temporar...
1  01 taken account help scammers testing account...
2  0123456789 1234567890 sit suburbs enjoy free s...
3  0200 day finished time way fake effort work bo...
4  03 months lot people receive clothes sizes don...

In [40]:
cvec3_df['feature3'].value_counts().tail()
cv33_df = pd.DataFrame(cvtrans3_text_train.todense(), columns = cvec3_df)

In [41]:
tfidf1 = TfidfVectorizer(stop_words='english', ngram_range=(2,2))  
tfidf2 = TfidfVectorizer(stop_words='english', ngram_range=(5,5))
tfidf3 = TfidfVectorizer(stop_words='english', ngram_range=(10,10))

In [49]:
tfidf1.fit(X_train)
trans1_text_train = tfidf1.transform(X_train)
trans1_text_test = tfidf1.transform(X_test)

In [50]:
tfidf1_df = pd.DataFrame(tfidf1.get_feature_names())
tfidf1_df.columns = ['feature1']
tfidf1_df.head()

feature1
0       000 followed
1           000 time
2  0000 wwwredditcom
3           001 work
4              01 10

In [52]:
tfidf1_xtrain_df = pd.DataFrame(trans1_text_train.todense(), columns = tfidf1_df)
#tfidf1_xtest_df = pd.DataFrame(trans1_text_test.todense(), columns = trans1_text_test.get_feature_names() )

In [53]:
tfidf2.fit(X_train)
tfidf2.transform(X_train)
tfidf2.transform(X_test)

<11988x279266 sparse matrix of type '<class 'numpy.float64'>'
	with 10146 stored elements in Compressed Sparse Row format>

In [92]:
tfidf2_df = pd.DataFrame(tfidf2.get_feature_names())
tfidf2_df.columns = ['feature2']
tfidf2_df.tail()

feature2
279261                        าหนาว เร มต นละ 877บาท
279262                       าเช ยงใหม ราคาถ อนร บหน
279263                   โปรโมช รถเช าเช ยงใหม ราคาถ
279264  六四天安門事件june 4th tiananmen square massacreand
279265                    𝐔𝐋𝐏𝐓if youre mad papa john

In [55]:
tfidf3.fit(X_train)
tfidf3.transform(X_train)
tfidf3.transform(X_test)

<11988x139169 sparse matrix of type '<class 'numpy.float64'>'
	with 3868 stored elements in Compressed Sparse Row format>

In [0]:
#tfidf_df = pd.DataFrame(transformed_text.todense(), columns = tfidf3.get_feature_names())

In [91]:
tfidf3_df = pd.DataFrame(tfidf3.get_feature_names())
tfidf3_df.columns = ['feature3']
tfidf3_df.head()

feature3
0  0000 wwwredditcom hosts file computer temporar...
1  01 taken account help scammers testing account...
2  0123456789 1234567890 sit suburbs enjoy free s...
3  0200 day finished time way fake effort work bo...
4  03 months lot people receive clothes sizes don...

In [57]:
tfidf3.fit(X_train)
trans3_text_train = tfidf3.transform(X_train)
trans3_text_test = tfidf3.transform(X_test)

tfidf33_df = pd.DataFrame(trans3_text_train.todense(), columns = tfidf3_df)

In [0]:
## YOUR CODE HERE

#cvec = CountVectorizer()          params = stop_words='english', ngrams= 2-5 #try      #has a plot inside it
#cvec.fit(scraped file)            on x_train, x_test
#cvec.transform(same file)

#cvec.get_feature_names
#cvec.get_feature_names_.sum()    #make a visualization of this 
 
    #decide if I'd like to use standard scaler - if there is a huge difference bt the num of counts
    #if I expand x beyond feature names, I will most likely have to scale.

#sns.barplot, scatterplot, or heatmap


#this process also needs to be done w tfidf - which will be more helpful bc we arelooking to discriminiate while 
#classifying 

#note: everything I do to my x_train, I must do to x_test

## Predicting subreddit using Random Forests + Another Classifier

In [64]:
## YOUR CODE HERE
#params = [{'n_estimators': 10,
           #'criterion':['gini','entropy'], 
           #'max_depth':[5, 10, 20], 
           #'warm_start':'True'}]

rf = RandomForestClassifier(n_estimators=10)

#x_train, x_test can be just the features

#first model with just vectorized features, then again using other features (numerical columns)

In [65]:
s = cross_val_score(rf, trans3_text_train, y_train, n_jobs=-1)

print("{} Score:\t{:0.3} ± {:0.3}".format("Random Forest", s.mean().round(3), s.std().round(3)))

Random Forest Score:	0.526 ± 0.001


#### Create a `RandomForestClassifier` model to predict which subreddit a given post belongs to.


In [66]:
rf1 = RandomForestClassifier()

#### We want to predict a binary variable - class `0` for one of your subreddits and `1` for the other.

#Your code here

This is the first cell in my Preprocessing Section.  Cell 16

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy of the model, as well as any other metrics you feel are appropriate. 
- **Bonus**: Use `GridSearchCV` with `Pipeline` to optimize your `CountVectorizer`/`TfidfVectorizer` and classification model.

In [67]:
# Number of trees in random forest
#n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
#

In [68]:
grid = {
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              'bootstrap': bootstrap}

In [69]:
rf_grid = GridSearchCV(estimator = rf1, param_grid = grid, cv = 5, verbose=2, n_jobs = -1)

In [70]:
rf_grid = rf_grid.fit(trans3_text_train, y_train)


Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[CV] bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2 
[CV] bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2 
[CV] bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2 
[CV] bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2 
[CV]  bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, total=   1.9s
[CV]  bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, total=   1.9s
[CV] bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2 
[CV] bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=5 
[CV]  bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, total=   1.9s
[CV] bootstrap=True, max_depth=10, m

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   16.1s


[CV]  bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5, total=   0.5s
[CV] bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5 
[CV]  bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5, total=   0.4s
[CV] bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5 
[CV]  bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5, total=   0.5s
[CV] bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5 
[CV]  bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5, total=   0.5s
[CV] bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=10 
[CV]  bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=10, total=   2.8s
[CV] bootstrap=True, max_depth=10, max_features=auto, min_samples_le

[CV]  bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, total=   2.8s
[CV] bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2 
[CV]  bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, total=   0.4s
[CV] bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2 
[CV]  bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, total=   0.4s
[CV] bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2 
[CV]  bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, total=   0.4s
[CV] bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=5 
[CV]  bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, total=   0.3s
[CV] bootstrap=True, max_depth=10, max_features=sqrt, min_samples_lea

[CV]  bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=5, total=   6.1s
[CV] bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=10 
[CV]  bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=10, total=   5.1s
[CV] bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=10 
[CV]  bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=5, total=   5.5s
[CV] bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=10 
[CV]  bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=10, total=   5.0s
[CV]  bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=10, total=   5.8s
[CV] bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV] bootstrap=True, max_depth=20, max_features=auto, min_sample

...

[CV] bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=4, min_samples_split=10 
[CV]  bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=4, min_samples_split=5, total=   0.4s
[CV] bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=4, min_samples_split=10 
[CV]  bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=4, min_samples_split=10, total=   0.4s
[CV] bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=4, min_samples_split=10 
[CV]  bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=4, min_samples_split=10, total=   0.4s
[CV] bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=4, min_samples_split=10 
[CV]  bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=4, min_samples_split=10, total=   0.4s
[CV] bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=4, min_samples_split=10 
[CV]  bootstrap=False, max_depth=110, max_features=auto, mi

[CV]  bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, total=   0.7s
[CV] bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=4, min_samples_split=5 
[CV]  bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, total=   0.8s
[CV] bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=4, min_samples_split=5 
[CV]  bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, total=   0.7s
[CV] bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=4, min_samples_split=5 
[CV]  bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, total=   0.9s
[CV] bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=4, min_samples_split=5 
[CV]  bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, total=   0.7s
[CV] bootstrap=False, max_depth=110, max_features=sq

[CV]  bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=10, total= 1.3min
[CV] bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=10 
[CV]  bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=10, total= 1.6min
[CV] bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV]  bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=2, total=   0.4s
[CV] bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV]  bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=2, total=   0.4s
[CV] bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV]  bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=2, total=   0.4s
[CV] bootstrap=False, max_depth=None, ma

[CV]  bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, total=12.0min
[CV] bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5 
[CV]  bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, total= 2.2min
[CV] bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5 
[CV]  bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, total= 1.8min
[CV] bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=10 
[CV]  bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, total= 1.8min
[CV] bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=10 
[CV]  bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, total= 1.5min
[CV] bootstrap=False, max_depth=None, ma

[Parallel(n_jobs=-1)]: Done 2160 out of 2160 | elapsed: 569.1min finished


In [71]:
rf_grid.best_score_
rf_grid.best_params_
best_rf = rf_grid.best_estimator_
best_rf.score(trans3_text_test, y_test)

0.5382048715382048

In [89]:
print(rf_grid.best_score_)
print(rf_grid.best_params_)

0.5331460986596963
{'bootstrap': False, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10}


#### Thought experiment: What is the baseline accuracy for this model?

In [0]:
## YOUR CODE HERE

In [0]:
50%. This accuracy score is based on the mean of the target variable. In this case, half of the the target came from 
subreddit and the other half came from the other subreddit.

#### Repeat the model-building process using a different classifier (e.g. `MultinomialNB`,   `LogisticRegression`, etc)

## LogisticRegression

In [74]:
from sklearn.linear_model import LogisticRegression


In [80]:
#logreg
X2 = trans3_text_train
y2 = y_train

#X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2) 

logreg = LogisticRegression()
logreg.fit(X2, y2_train)

print('Intercept:', logreg.intercept_)
print('Coef(s):', logreg.coef_)



#create a dataframe with the predictions and the actual. 


Intercept: [0.00308054]
Coef(s): [[ 0.16348764 -0.12701434  0.1334871  ... -0.23189539 -0.23189539
  -0.20196555]]


In [ ]:
print('Logreg predicted class:', logreg.predict(X2))
print('Logreg predicted probabilities:', logreg.predict_proba(X2))

#LifeProTips is the positive class.

In [87]:
score = logreg.score(trans3_text_test, y_test)


In [88]:
score

0.5130964297630964

# Support Vector Machines

In [ ]:
X3 = reddit['work']
y3 = reddit['target']

X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3) 

In [ ]:
from sklearn import svm, linear_model
from sklearn.metrics import accuracy_score

In [ ]:
svc = svm.SVC()


In [ ]:
svc.fit(X3_train, y3_train)

y_predictions = svc.predict(X3_test)

accuracy_score(y3_test, y_prections)

In [ ]:
results = pd.DataFrame(svc.predict(X3_test), columns = ['predicted'])
results['true'] = y3_test


results.head(30)

# Executive Summary
---
Put your executive summary in a Markdown cell below.

Type Markdown and LaTeX:  α2

Be Ethical!   #Ethical or Nah?

Opener & Need:
  Reddit is a platform that offers a myraid of advice, entertainment and fluff about everything from how-to-vids 
to sports. "Be ethical" is aproduct constructed using the machine learning algorithms natural language processing, 
RandomForestClassifiers and KNNClassifiers.

Solution:
  "Be ethical" allows you to feel safe when your teens browse reddit. It accurately predicts which subreddits are
(in this case) unethical, thus protecting your children from mayhem.

Evidence:
  Over the past two months I have dedicated nearly every waking hour to understanding the mechanics of machine 
learning. I was recently challenged to find the best way to categorize two subreddits. My colleagues and I 
designed a web scraper, a tool designed to capture raw data directly from two different subreddits. Titles from 
these subreddits were preprocessed and vectorized, using CountVectorizer and TfidfVectorizer,to convert actual 
text to numerical values that were then used as features in classification models. Both CountVectorizer and Tfidf
are sklearn feature extraction methods used to create determine the frequency and 'importance'of tokens in our
dataset.  The classification models then predicted which subreddit each 
title came from.

First, the data was scored using 3-Fold crossvalidation through a RandomForest with no hypertuning. The score was 
abysmal, suggesting RandomForest's classification strategy is not the best for this problem. OF the classification
and regression tree (CART) toolkit a regression model may provide more accuracy. But before moving on to logistic
regression model, hypertuning was in order to ensure no stone was left unturned.

So, an extensive grid search was conducting using 5-fold cross-validation to determine the optimal parameters 
and hyperparaters for the RandomForestClassifier. 
The functions .best_score_, .best_params_, and best_estimator_ select from those provided by the scientist
the best parameters for modeling.

Powerful Processing::This process is computationally taxing and crashed my cpu several times. It was down for several 
hours - resulting in the discovery of Google's colab. 

Logistic Regression was employed to estimate the class of subreddit. This model is appealing because it minimizes deviance and the lack of closed-form solutions means the betas are found through optimization of the logit function.

The baseline accuracy was 50%.  The corpus, which is the entire collection of scrapped and processed subreddit 
titles, was nearly evenly split between the two subreddits, LifeProTips and UnethicalLifeProTips.
Results showed the best model was the RandomForest GridSearch which returned a score of .54 with .  Logistic Regression returned a a 51% accuracy score. GridSearch determined the best parameters for the RandomForest were {bootstrap': False, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10}.

Improvements: Test the RandomForest with each sub of CounVectorized data and each sub of TfidfVectorized data to see which performs best. The time of posts will also likely provide useful information in predicting class. I suspect UnethicalLifeProTips are posted at discernably different times of day.

Learning:  One thing I did not do was visualize feature importances for my vectorizers. That would have allowed me to better select features to include in my model.

Finally, a support vector machine classifier was selected to test for non-linear separation between the classes. This blackbox model allows gridsearching and includes what is called the kernel trick. The complexity of the math disallows easily relatable explanation about how it works. Nonetheless, this model's ability to discriminate on 
